# EXPERIMENTING

In [1]:
import dolfinx, sys, petsc4py
print("dolfinx version:", dolfinx.__version__)
print("dolfinx path:", dolfinx.__file__)
print("Python:", sys.version)
print("PETSc4py:", petsc4py.__version__)

dolfinx version: 0.9.0
dolfinx path: /home/ug/miniconda3/envs/fenicsx/lib/python3.13/site-packages/dolfinx/__init__.py
Python: 3.13.9 | packaged by conda-forge | (main, Oct 16 2025, 10:31:39) [GCC 14.3.0]
PETSc4py: 3.24.0


In [2]:
# --- Enable threaded PETSc linear algebra inside Jupyter ---
from petsc4py import PETSc
import os

# Tell PETSc and OpenBLAS/MKL to use all CPU cores
os.environ["OMP_NUM_THREADS"] = "24"     # OpenMP threads for BLAS/LAPACK
os.environ["MKL_NUM_THREADS"] = "24"     # if MKL is used
os.environ["OPENBLAS_NUM_THREADS"] = "24"

# PETSc internal thread count
PETSc.Options()["openmp_num_threads"] = 24

# Optional: avoid duplicate error printing
PETSc.Sys.popErrorHandler()

# Verify
print(f"✅ PETSc threading set to {PETSc.Options().getInt('openmp_num_threads', -1)} threads")

✅ PETSc threading set to 24 threads


Bring in needed dependencies

In [3]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot

from dolfinx.nls.petsc import NewtonSolver
from petsc4py import PETSc

from dolfinx.io import XDMFFile
import traceback
from lxml import etree

Define the domain

In [4]:
# --- Rubber block ---
rubber = mesh.create_box(MPI.COMM_WORLD,
    [[0, 0, 0], [10, 5, 7.5]],
    [20, 10, 10],
    mesh.CellType.hexahedron)

# --- Metal cube ---
metal = mesh.create_box(MPI.COMM_WORLD,
    [[2.5, 5, 2.5], [5, 7.5, 5]],
    [10, 4, 5],
    mesh.CellType.hexahedron)

# --- Save each separately ---
with XDMFFile(rubber.comm, "rubber.xdmf", "w") as xdmf:
    xdmf.write_mesh(rubber)

with XDMFFile(metal.comm, "metal.xdmf", "w") as xdmf:
    xdmf.write_mesh(metal)

print("✅ Saved rubber.xdmf and metal.xdmf — open both in ParaView together.")


✅ Saved rubber.xdmf and metal.xdmf — open both in ParaView together.


Create two python functions for determining the facets to apply boundary conditions to

In [5]:
# --- Rubber tags ---
fdim_r = rubber.topology.dim - 1

def bottom_r(x): return np.isclose(x[1], 0.0)
def top_r(x):    return np.isclose(x[1], 5.0)

bottom_facets = mesh.locate_entities_boundary(rubber, fdim_r, bottom_r)
top_facets    = mesh.locate_entities_boundary(rubber, fdim_r, top_r)

facet_values = np.hstack([np.full_like(bottom_facets, 1), np.full_like(top_facets, 10)])
facet_indices = np.hstack([bottom_facets, top_facets])
facet_tags_r = mesh.meshtags(rubber, fdim_r, facet_indices, facet_values)

# --- Metal tags ---
fdim_m = metal.topology.dim - 1

def bottom_m(x): return np.isclose(x[1], 5.0)
def top_m(x):    return np.isclose(x[1], 7.5)

bottom_facets_m = mesh.locate_entities_boundary(metal, fdim_m, bottom_m)
top_facets_m    = mesh.locate_entities_boundary(metal, fdim_m, top_m)

facet_values_m = np.hstack([np.full_like(bottom_facets_m, 20), np.full_like(top_facets_m, 21)])
facet_indices_m = np.hstack([bottom_facets_m, top_facets_m])
facet_tags_m = mesh.meshtags(metal, fdim_m, facet_indices_m, facet_values_m)

Next, create a  marker based on these two functions

Create a function for supplying the boundary condition on the left side, which is fixed.

In [6]:
V_r = fem.functionspace(rubber, ("Lagrange", 2, (rubber.geometry.dim,)))
u_r = fem.Function(V_r, name="Rubber_Displacement")
v_r = ufl.TestFunction(V_r)

V_m = fem.functionspace(metal, ("Lagrange", 1, (metal.geometry.dim,)))
u_m = fem.Function(V_m, name="Metal_Displacement")  # For visualization

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [7]:
u_bc = np.array((0.0, 0.0, 0.0))
bottom_dofs = fem.locate_dofs_topological(V_r, fdim_r, facet_tags_r.find(1))
bcs = [fem.dirichletbc(u_bc, bottom_dofs, V_r)]

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u_r)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u_r))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$,
and create the expression for the first Piola-Kirchhoff stress:

Elasticity parameters

In [9]:
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(rubber, E / (2 * (1 + nu)))
lmbda = fem.Constant(rubber, E * nu / ((1 + nu) * (1 - 2 * nu)))

Stored strain energy density (compressible neo-Hookean model)

In [10]:
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J)) ** 2

1st PK stress

In [11]:
P = ufl.diff(psi, F)

Cauchy stress

In [12]:
# --- Compute Cauchy stress symbolically ---
J = ufl.det(F)
sigma = (1.0 / J) * P * F.T  # σ = (1/J) * P * Fᵀ

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity,
the following lines can be uncommented to solve the linear elasticity problem.
```

In [13]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2.
We set the quadrature degree for the integrals to 4.

In [14]:
epsilon_c = fem.Constant(rubber, 1e5)
lambda_N = fem.Function(fem.functionspace(rubber, ("DG", 0)))

n_r = ufl.FacetNormal(rubber)

# Gap function: (rigid metal moves down by delta)
delta = fem.Constant(rubber, 0.2)
gN = (5.0 - delta) - (ufl.SpatialCoordinate(rubber)[1] + u_r[1])  # rubber top to rigid plane at y=5−δ
pen = ufl.max_value(0.0, -gN)

tN = lambda_N + epsilon_c * pen

ds_contact = ufl.Measure("ds", domain=rubber, subdomain_data=facet_tags_r, subdomain_id=10)
R_contact = ufl.inner(tN * n_r, v_r) * ds_contact

Define the residual of the equation (we want to find u such that residual(u) = 0)

In [15]:
dx_r = ufl.Measure("dx", domain=rubber)
B = fem.Constant(rubber, np.array((0.0, 0.0, 0.0), dtype=np.float64))

residual = ufl.inner(ufl.grad(v_r), P) * dx_r - ufl.inner(v_r, B) * dx_r + R_contact

As the varitional form is non-linear and written on residual form,
we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [16]:
log.set_log_level(log.LogLevel.INFO)

# 1️⃣ Define nonlinear problem
problem = NonlinearProblem(residual, u_r, bcs=bcs)

# 2️⃣ Configure PETSc options globally
opts = PETSc.Options()
opts["snes_type"] = "newtonls"
opts["snes_linesearch_type"] = "bt"
opts["ksp_type"] = "preonly"
opts["pc_type"] = "lu"
opts["pc_factor_mat_solver_type"] = "petsc"
# You can add monitors if desired:
# opts["snes_monitor"] = ""

# 3️⃣ Create solver and tune tolerances
solver = NewtonSolver(u_r.function_space.mesh.comm, problem)
solver.rtol = 1e-6
solver.atol = 1e-6
solver.stol = 1e-8
solver.max_it = 50
solver.report = True
solver.convergence_criterion = "incremental"

[2025-11-02 07:16:41.568] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-11-02 07:16:41.568] [info] Requesting connectivity (3, 0) - (2, 0)
[2025-11-02 07:16:41.568] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-11-02 07:16:41.568] [info] Requesting connectivity (3, 0) - (2, 0)
[2025-11-02 07:16:41.571] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-11-02 07:16:41.571] [info] Requesting connectivity (3, 0) - (2, 0)
[2025-11-02 07:16:41.571] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-11-02 07:16:41.571] [info] Requesting connectivity (3, 0) - (2, 0)
[2025-11-02 07:16:41.572] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-11-02 07:16:41.591] [info] Column ghost size increased from 0 to 0


Generic projection definition

In [17]:
# --- Generic projection helper ---
def project(expr, V, name=None):
    """
    Project a UFL expression 'expr' into function space V.
    Returns a fem.Function containing the projected field.
    """
    v = ufl.TestFunction(V)
    t = ufl.TrialFunction(V)
    a = ufl.inner(t, v) * ufl.dx
    L = ufl.inner(expr, v) * ufl.dx
    problem = fem.petsc.LinearProblem(a, L)
    f = fem.Function(V, name=name or "projection")
    f_sol = problem.solve()
    f.x.array[:] = f_sol.x.array
    return f

Finally, we solve the problem over several time steps, updating the z-component of the traction

In [18]:
from dolfinx import io

# --- Output spaces once ---
V_out = fem.functionspace(rubber, ("Lagrange", 1, (rubber.geometry.dim,)))
W_CG  = fem.functionspace(rubber, ("Lagrange", 1, (rubber.geometry.dim, rubber.geometry.dim)))
W_DG  = fem.functionspace(rubber, ("Discontinuous Lagrange", 0, (rubber.geometry.dim, rubber.geometry.dim)))

# --- Output function containers ---
u_out = fem.Function(V_out, name="Displacement")

# --- Open XDMF file ---
with io.XDMFFile(rubber.comm, "rubber_contact_results.xdmf", "w") as xdmf:
    xdmf.write_mesh(rubber)
    xdmf.write_function(u_out, t=0.0)
    xdmf.write_function(fem.Function(W_CG, name="Cauchy_stress_CG"), t=0.0)
    xdmf.write_function(fem.Function(W_DG, name="Cauchy_stress_DG"), t=0.0)
    xdmf.write_function(fem.Function(W_CG, name="Green_Lagrange_strain_CG"), t=0.0)
    xdmf.write_function(fem.Function(W_DG, name="Green_Lagrange_strain_DG"), t=0.0)

    # indentation steps
    for n, dval in enumerate(np.linspace(0.0, 0.5, 10)):
        print(f"\n=== Step {n}: indentation {dval:.3f} ===")
        delta.value = dval

        try:
            n_iter, converged = solver.solve(u_r)
            print(f"Step {n}: {'✅ converged' if converged else '❌ not converged'} in {n_iter} iterations")
        except Exception as e:
            print(f"❌ Solver crashed at step {n}: {e}")
            break

        # --- write displacement field ---
        u_out.interpolate(u_r)
        xdmf.write_function(u_out, t=float(n))

        # --- Cauchy stress ---
        I = ufl.Identity(rubber.geometry.dim)
        F = I + ufl.grad(u_r)
        P = ufl.diff(psi, F)
        sigma = (1.0 / ufl.det(F)) * P * F.T  # σ = (1/J) P Fᵀ

        sigma_CG = project(sigma, W_CG, name="Cauchy_stress_CG")
        xdmf.write_function(sigma_CG, t=float(n))
        sigma_DG = project(sigma, W_DG, name="Cauchy_stress_DG")
        xdmf.write_function(sigma_DG, t=float(n))

        # --- Green–Lagrange strain ---
        E_expr = 0.5 * (F.T * F - I)
        E_CG = project(E_expr, W_CG, name="Green_Lagrange_strain_CG")
        E_DG = project(E_expr, W_DG, name="Green_Lagrange_strain_DG")
        xdmf.write_function(E_CG, t=float(n))
        xdmf.write_function(E_DG, t=float(n))

        # --- contact multiplier output (optional) ---
        xdmf.write_function(lambda_N, t=float(n))

        print(f"📤 Saved displacement, stresses, and strains for step {n}")

        if np.isnan(u_r.x.array).any():
            print("⚠️ NaN detected in solution, stopping loop.")
            break

print("✅ Simulation completed successfully.")

[2025-11-02 07:16:41.605] [info] Checking required entities per dimension

=== Step 0: indentation 0.000 ===
[2025-11-02 07:16:41.605] [info] Cell type: 0 dofmap: 2000x8
[2025-11-02 07:16:41.605] [info] Global index computation
[2025-11-02 07:16:41.605] [info] Got 1 index_maps
[2025-11-02 07:16:41.605] [info] Get global indices
[2025-11-02 07:16:41.606] [info] Checking required entities per dimension
[2025-11-02 07:16:41.606] [info] Cell type: 0 dofmap: 2000x8
[2025-11-02 07:16:41.606] [info] Global index computation
[2025-11-02 07:16:41.606] [info] Got 1 index_maps
[2025-11-02 07:16:41.606] [info] Get global indices
[2025-11-02 07:16:41.608] [info] Checking required entities per dimension
[2025-11-02 07:16:41.608] [info] Cell type: 0 dofmap: 2000x1
[2025-11-02 07:16:41.608] [info] Global index computation
[2025-11-02 07:16:41.608] [info] Got 1 index_maps
[2025-11-02 07:16:41.608] [info] Get global indices
[2025-11-02 07:16:41.609] [info] Opened HDF5 file with id "72057594037927938"
[2

IOStream.flush timed out
IOStream.flush timed out


[2025-11-02 07:20:06.402] [info] PETSc Krylov solver starting to solve system.
[2025-11-02 07:20:39.411] [info] Newton iteration 2: r (abs) = 2.7843124075577567e-14 (tol = 1e-06), r (rel) = 1.225586557916833 (tol = 1e-06)
[2025-11-02 07:20:39.411] [info] Newton solver finished in 2 iterations and 2 linear solver iterations.
Step 0: ✅ converged in 2 iterations
[2025-11-02 07:20:39.421] [info] Adding function to node "/Xdmf/Domain/Grid/Grid"


HDF5-DIAG: Error detected in HDF5 (1.14.6) MPI-process 0:
  #000: H5D.c line 186 in H5Dcreate2(): unable to synchronously create dataset
    major: Dataset
    minor: Unable to create file
  #001: H5D.c line 135 in H5D__create_api_common(): unable to create dataset
    major: Dataset
    minor: Unable to create file
  #002: H5VLcallback.c line 1865 in H5VL_dataset_create(): dataset create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #003: H5VLcallback.c line 1830 in H5VL__dataset_create(): dataset create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #004: H5VLnative_dataset.c line 282 in H5VL__native_dataset_create(): unable to create dataset
    major: Dataset
    minor: Unable to initialize object
  #005: H5Dint.c line 351 in H5D__create_named(): unable to create and link to dataset
    major: Dataset
    minor: Unable to initialize object
  #006: H5Lint.c line 492 in H5L_link_object(): unable to create new link to object
    m

RuntimeError: Failed to create HDF5 global dataset.